<a href="https://colab.research.google.com/github/junseokShim/Agents_from_LLMS/blob/main/dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
# 디바이스 설정 (M3 칩에서 MPS 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device.type)

# 모델 로드 (경량 LLM, MPS 호환)
model_name = "nlpai-lab/kullm-polyglot-13.6b"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2

# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

# 인터넷 검색을 통한 추가 정보 수집 함수
def web_search(query):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        search_url = f"https://www.google.com/search?q={query}"
        response = requests.get(search_url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        snippets = soup.select('div.BNeawe.s3v9rd.AP7Wnd')
        results = ' '.join(snippet.get_text() for snippet in snippets[:3])
        return results
    except requests.exceptions.RequestException:
        return ""

# 프롬프트 생성기
def make_prompt_auto(text):
    additional_info = web_search(text)

    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
            "당신은 금융보안 전문가입니다. 제공된 추가 정보와 질문을 고려하여 가장 적절한 정답 번호만 간단히 제시하세요.\n\n"
            f"추가 정보: {additional_info}\n\n"
            f"질문: {question}\n"
            "선택지:\n"
            f"{chr(10).join(options)}\n\n"
            "정답 번호:"
        )
    else:
        prompt = (
            "당신은 금융보안 전문가입니다. 제공된 추가 정보를 참고하여 아래 질문에 간결하고 명료한 답변을 제공하세요.\n\n"
            f"추가 정보: {additional_info}\n\n"
            f"질문: {text}\n\n"
            "답변:"
        )
    return prompt

# 로컬 모델을 통한 답변 생성 함수 (직접 generate 사용)
def get_local_model_response(prompt, max_new_tokens=300):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text[len(prompt):].strip()

# 예시 활용 코드
def generate_answer_from_text(text):
    prompt = make_prompt_auto(text)
    response = get_local_model_response(prompt)
    return response


In [ ]:
if __name__ == "__main__":
    # 데이터 로드
    test = pd.read_csv('./data/test.csv')

    # 데이터프레임에 적용
    answers = []
    for idx, row in tqdm(test.iterrows(), total=len(test)):
        answer = generate_answer_from_text(row['Question'])
        answers.append(answer)

    test['generated_answer'] = answers

    # 결과 확인 및 저장
    sample_submission = pd.read_csv('./data/sample_submission.csv')
    sample_submission['Answer'] = answers
    sample_submission.to_csv('./submission_gpu.csv', index=False, encoding='utf-8-sig')